In [1]:
# TODO: Read files: image (TDET + POCR + VOCR), pdf, excel, docx, text
# TODO: Combine TDET + POCR + VOCR -> VDOCR
# TODO: Put VDOCR into gradio app

In [2]:
from pkg.VDOCR.TDET.TDET import Process_TDET
from pkg.VDOCR.POCR.POCR import Process_POCR
from pkg.VDOCR.VOCR.VOCR import Process_VOCR
import pkg.UTILS.UTILS as UTILS
import cv2

In [3]:
img_ocv = cv2.imread("_test/img_17.jpg")
img_ocv = UTILS.preprocess_document_image(img_ocv)

In [4]:
tables_bboxes = Process_TDET(img_ocv)
texts_bboxes = Process_POCR(img_ocv, padding_ratio=0.2)

In [ ]:
# text = Process_VOCR("_test/img_18.jpg")